In [ ]:
import dlib
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import numpy as np
import pandas as pd
import os
import requests
import json

### 이미지 크롤링

In [ ]:
working_directory = '/Users/leslie/Documents/Big_Project/image_crwal_using_kakao_api/'
os.chdir(working_directory)
os.getcwd()

'/Users/leslie/Documents/Big_Project/image_crwal_using_kakao_api'

In [ ]:
query = input("검색어 입력 : ")
dir_name = input("디렉토리명 입력 : ")

검색어 입력 :  세븐틴 도겸
디렉토리명 입력 :  가을_뮤트_도겸


In [ ]:
!mkdir {dir_name}

def save_image(image_url, file_name, dir_name):
    img_response = requests.get(image_url)
    # 요청에 성공했다면,
    if img_response.status_code == 200:
        # 파일 저장
        full_name = dir_name+"/"+file_name
        if img_response.content[0:2]!=bytes(b'\xff\xd8') : return # jpg파일인지 binary check
        with open(full_name, "wb") as fp:
            fp.write(img_response.content)

# 이미지 검색 
url = "https://dapi.kakao.com/v2/search/image"
headers = {
    "Authorization" : "KakaoAK MYAPICODE"
}

count = 0
# 1페이지에 80장, 최대 50페이지 가능
page = 50
for p in range(1,page+1):
  print(f'===================={p}====================')
  data = {
      "query" : query,
      "page" : p,
  }

  # 이미지 검색 요청
  response = requests.post(url, headers=headers, data=data)
  # 요청에 실패했다면,
  if response.status_code != 200:
      print("error! because ",  response.json())
  else: # 성공했다면,
      for image_info in response.json()['documents']:
          print(f"[{count}th] image_url =", image_info['image_url'])
          count = count + 1
          count_str = str(count).zfill(5)
          file_name = f'{dir_name}_{count_str}.jpg'
          # 이미지 저장
          save_image(image_info['image_url'], file_name, dir_name)

### 이미지 데이터클렌징, 얼굴크롭

In [ ]:
dir_name = '가을_뮤트_도겸'
os.chdir(working_directory+dir_name)
print('Current Working Dir : '+os.getcwd())
imgs = os.listdir(os.getcwd())
imgs = [img for img in imgs if 'jpg' in img]
imgs.sort()
imgs[0], imgs[1], imgs[-1], len(imgs)

In [ ]:
print(os.getcwd(), '\n', imgs[0], '\n', imgs[-1])
os.makedirs('cropped', exist_ok=True)
os.makedirs('masked', exist_ok=True)

In [ ]:
face_detector = dlib.get_frontal_face_detector()
idx = 0
lower = np.array([0,133,77], dtype = np.uint8)
upper = np.array([255,173,127], dtype = np.uint8)
for image_path in imgs :
    img = cv2.imread(image_path)
    faces = face_detector(img)
    print(image_path)
    # 얼굴 인식이 하나가 아닐때 해당 파일 삭제
    if len(faces) != 1 :
        idx += 1
        print(str(idx)+image_path+' 얼굴인식 수 : '+str(len(face_detector(img))))
        os.remove(image_path)
    # 얼굴 영역만 크롭하여 cropped 폴더에 저장
    else :
        # 결과 출력 & 이미지에 해당 검출 부분들을 빨간색(bgr 순) 박스로 표시 (굵기 2)
        # print("left, top. right, bottom : ", f.left(), f.top(), f.right(), f.bottom())
        # cv2.rectangle(img, (f.left(), f.top()), (f.right(), f.bottom()), (0,0,255), 2)
        f = faces[0]
        cropped_img = img[f.top()+2:f.bottom()-2, f.left()+2:f.right()-2]
        if cropped_img.size==0 : continue
        cv2.imwrite(f'./cropped/{image_path}',cropped_img)        

### 마스킹 코드

In [ ]:
lower = np.array([0,133,77], dtype = np.uint8)
upper = np.array([255,173,127], dtype = np.uint8)

# input test dir_name
dir_name = '가을_뮤트_도겸'
os.chdir(working_directory+dir_name+'/cropped')
cropped_list = os.listdir(os.getcwd())
cropped_list = [img for img in cropped_list if 'jpg' in img]
for c_photo_file_name in cropped_list :
  cropped_img = cv2.imread(c_photo_file_name)
  if cropped_img.size==0 : continue
  
  # 크롭된 이미지를 마스킹하여 저장
  face_img_ycrcb = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2YCrCb)
  skin_msk = cv2.inRange(face_img_ycrcb, lower, upper)
  skin = cv2.bitwise_and(cropped_img, cropped_img, mask = skin_msk)
  cv2.imwrite(f'../masked/{c_photo_file_name}',skin)


In [ ]:
# 검은 영역이 전체 20?이상되면 버리게하는 코드

skin = cv2.imread(c_photo_file_name)

h, w, rgb = skin.shape
low = (0,0,0)
high = (0,0,0)
mask = cv2.inRange(skin, low, high)
mask = 255 - mask

# find black coordinates
coords = np.argwhere(mask==0)
print(len(coords))
print((len(coords)/(h*w))*100)

2819
13.22480765622068
